In [5]:
import os
from bids import BIDSLayout

In [6]:
bids_dir = "../data"

In [7]:
layout = BIDSLayout(bids_dir)

In [1]:
base_dir = "/projects/mjoseph/pipelines/testing/data"

- GE/Philips: EchoTrainLength it will be Echo train length/PE steps  
- SIEMENS: EPIFactor 
acquisition direction A>>P or P>>A

total_readout_time = 1 / PE bandwidth  
effective echo spacing = 1 / (PE bandwidth * (ETL - 1) * acc factor)

- echo spacing = .53ms
- number of in plane lines in the reconstructed epi image = 64
- acceleration factor = 2
- number of in plane reference lines = 24

My questions: 
What do I have to enter as the echo spacing for b0 field unwarping with 
feat/fugue?
(a) .53ms/2 (echo_spacing*#lines_in_images / acceleration factor) 
(b) .53*64/(32+24) (echo_spacing*#lines_in_images/(#lines_in_images + 
#reference_lines))
(c) something else ...

In [5]:
from bids import BIDSLayout

layout = BIDSLayout(base_dir)

In [4]:
dwi_file = "{}/sub-CMP0178/ses-01/dwi/sub-CMP0178_ses-01_acq-singleshell_dwi.nii.gz"

In [7]:
metadata = layout.get_metadata(dwi_file)

In [8]:
metadata

{}

In [1]:
def read_nifti_sidecar(path):
    pth, fname, ext = split_filename(path)
    json_file = op.join(pth, fname) + ".json"
    with open(json_file, "r") as f:
        metadata = json.load(f)
    pe_dir = metadata['PhaseEncodingDirection']
    slice_times = metadata.get("SliceTiming")
    trt = metadata.get("TotalReadoutTime")
    if trt is None:
        pass

    return {"PhaseEncodingDirection": pe_dir,
            "SliceTiming": slice_times,
            "TotalReadoutTime": trt}

In [2]:
acqp_lines = {
    "i": '1 0 0 %.6f',
    "j": '0 1 0 %.6f',
    "k": '0 0 1 %.6f',
    "i-": '-1 0 0 %.6f',
    "j-": '0 -1 0 %.6f',
    "k-": '0 0 -1 %.6f'}

In [3]:
def eddy_inputs_from_dwi_files(origin_file_list, dwi_files, eddy_prefix):
    unique_files = list(set(origin_file_list))
    line_lookup = {}
    acqp_data = []
    for line_num, unique_dwi in enumerate(unique_files):
        spec = read_nifti_sidecar(unique_dwi)
        spec_line = acqp_lines[spec['PhaseEncodingDirection']]
        acqp_line = spec_line % spec['TotalReadoutTime']
        line_lookup[unique_dwi] = line_num + 1
        acqp_data.append(acqp_line)

    # Create the acqp file
    acqp_file = eddy_prefix + "acqp.txt"
    with open(acqp_file, "w") as f:
        f.write("\n".join(acqp_data))

    # Create the index file
    index_file = eddy_prefix + "index.txt"
    index_numbers = [line_lookup[dwi_file] for dwi_file in origin_file_list]
    with open(index_file, "w") as f:
        f.write(" ".join(map(str, index_numbers)))

    return acqp_file, index_file

In [4]:
def topup_inputs_from_dwi_files(dwi_file_list, b0_file_list, topup_prefix, cwd, max_per_spec=3):
    """Create a datain spec and a slspec from a list of dwi files."""
    unique_files = list(set(dwi_file_list))
    spec_lookup = {}
    slicetime_lookup = {}
    for unique_dwi in unique_files:
        spec = read_nifti_sidecar(unique_dwi)
        spec_line = acqp_lines[spec['PhaseEncodingDirection']]
        spec_lookup[unique_dwi] = spec_line % spec['TotalReadoutTime']
        slicetime_lookup[unique_dwi] = spec['SliceTiming']

    # Write the datain.txt file
    datain_lines = []
    imain_images = []
    image_data = []
    spec_counts = defaultdict(int)

    def atleast4d(data):
        if data.ndim == 4:
            return data
        if data.ndim == 3:
            return data[:, :, :, np.newaxis]
        raise Exception("Less than 3 dimensions in b0 image")

    for dwi_file, b0_file in zip(dwi_file_list, b0_file_list):
        img = nb.load(b0_file)
        line = spec_lookup[dwi_file]
        num_trs = 1 if len(img.shape) < 4 else img.shape[3]
        available_slots = max_per_spec - spec_counts[line]

        if available_slots <= 0:
            continue

        if available_slots >= num_trs:
            datain_lines.extend([line] * num_trs)
            spec_counts[line] += num_trs
            imain_images.append(b0_file)
            image_data.append(atleast4d(img.get_fdata()))
        else:
            # Too many images for this spec
            num_to_add = available_slots
            truncated_image = fname_presuffix(b0_file, newpath=cwd, suffix="truncated")
            orig_img = nb.load(b0_file)
            LOGGER.warning("Truncating %s to %d volumes", b0_file, num_to_add)
            nb.Nifti1Image(atleast4d(orig_img.get_fdata())[:, :, :, :num_to_add],
                           orig_img.affine, orig_img.header).to_filename(truncated_image)
            imain_images.append(truncated_image)
            datain_lines.extend([line] * num_to_add)
            spec_counts[line] += num_to_add

    # Make a 4d series
    images = [nb.load(img) for img in imain_images]
    image_data = [img.get_fdata()[..., np.newaxis] if len(img.shape) == 3 else img.get_fdata()
                  for img in images]
    imain_output = topup_prefix + "imain.nii.gz"
    imain_img = nb.Nifti1Image(np.concatenate(image_data, 3), images[0].affine, images[0].header)
    assert imain_img.shape[3] == len(datain_lines)
    imain_img.to_filename(imain_output)

    # Write the datain text file
    datain_file = topup_prefix + "datain.txt"
    with open(datain_file, "w") as f:
        f.write("\n".join(datain_lines))

    # Check the slicetiming files
    return datain_file, imain_output
